In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

# Helper Functions

In [2]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')

def get_eye_info(i): return ejson(i, 'appleLeftEye.json'), ejson(i, 'appleRightEye.json')
def get_face_info(i): return ejson(i, 'appleFace.json')
def get_facegrid(i): return ejson(i, 'faceGrid.json')

def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords
def orientation_data(p):
    sdata = screen_data(p)
    return sdata['Orientation']

In [3]:
# dset_path = Path('/media/brennan/Data/ml/gazecapture/')
dset_path = Path('../gazecapture/')

# Extract File Names
We first split on case before splitting on frames

In [4]:
test_size = 0.1

In [5]:
invalid_cases = ['02984',
                 '02032',
                 '01030',
                 '01152',
                 '00325',
                 '02622',
                 '00827',
                 '02119',
                 '00927',
                 '03277',
                 '02190',
                 '01282',
                 '00808',
                 '01474',
                 '01805',
                 '00207',
                 '00666',
                 '01066',
                 '01243',
                 '01269',
                 '01661',
                 '02048',
                 '01266',
                 '00619',
                 '00779',
                 '01090',
                 '03059',
                 '01432',
                 '01225',
                 '03060',
                 '02781',
                 '00138',
                 '02526',
                 '01353',
                 '02967',
                 '02093',
                 '02165']

In [18]:
def extract_data(cases, accepted_o=[1, 2, 3, 4]):
    fnames = []
    XCam, YCam = [], []
    FaceH, FaceW, FaceX, FaceY = [], [], [], []
    IsValid = []
    
    for case in cases:
        if not (case in invalid_cases):
            FRAME_N = frame_data(case)
            O = orientation_data(case)
            XCAM, YCAM = coordinate_data(case)
            FACE = get_face_info(case)

            for frame_n, o, xcam, ycam, \
                fh, fw, fx, fy, valid in zip(FRAME_N, O, XCAM, YCAM, 
                                     FACE['H'], FACE['W'], FACE['X'], FACE['Y'], FACE['IsValid']):
                
                if o in accepted_o and valid == 1:
                    fnames.append('{}/frames/{}'.format(case.name, frame_n))
                    XCam.append(xcam)
                    YCam.append(ycam)
                    IsValid.append(valid)
                    FaceH.append(fh)
                    FaceW.append(fw)
                    FaceX.append(fx)
                    FaceY.append(fy)

    # package to dataframe
    df = pd.DataFrame(data={'file_names': fnames, 
                            'XCam': XCam,
                            'YCam': YCam,
                            'IsValid': IsValid,
                            'FaceH': FaceH, 
                            'FaceW': FaceW,
                            'FaceX': FaceX,
                            'FaceY': FaceY})        
    
    return df

# Extract all case names first to split => faces it hasnt seen before
train_cases, test_cases = train_test_split(list(dset_path.iterdir()), test_size=test_size)
len(train_cases), len(test_cases)

(1326, 148)

# Extract all Images

In [23]:
# create dataframe from data
# ALL PHOTOS split into train / test sets
train_df = extract_data(train_cases)
test_df = extract_data(test_cases)

train_df.to_csv('traindf.csv')
test_df.to_csv('testdf.csv')

# Extract ONLY Portrait Images

In [19]:
train_df = extract_data(train_cases, accepted_o=[1])
test_df = extract_data(test_cases, accepted_o=[1])
train_df.to_csv('portrait-data/traindf.csv')
test_df.to_csv('portrait-data/testdf.csv')

In [30]:
len(test_df), len(train_df)

(82893, 693269)

In [31]:
len(test_df) / (len(test_df) + len(train_df)), len(train_df) / (len(test_df) + len(train_df))

(0.10679858070866649, 0.8932014192913336)

# Extract & Split LandScape Images

In [20]:
train_df = extract_data(train_cases, accepted_o=[3])
test_df = extract_data(test_cases, accepted_o=[3])
train_df.to_csv('landscape-r-data/traindf.csv')
test_df.to_csv('landscape-r-data/testdf.csv')

In [21]:
train_df = extract_data(train_cases, accepted_o=[4])
test_df = extract_data(test_cases, accepted_o=[4])
train_df.to_csv('landscape-l-data/traindf.csv')
test_df.to_csv('landscape-l-data/testdf.csv')

# Extract Split Which Was Used In The Paper
http://gazecapture.csail.mit.edu/cvpr2016_gazecapture.pdf

In [13]:
class Dataset:
    def __init__(self):
        self.fnames = []
        self.XCam = []
        self.YCam = []
        self.FaceH, self.FaceW, self.FaceX, self.FaceY = [], [], [], []
    
    def append(self, fname, xcam, ycam, fh, fw, fx, fy):
        self.fnames.append(fname)
        self.XCam.append(xcam)
        self.YCam.append(ycam)
        self.FaceH.append(fh)
        self.FaceW.append(fw)
        self.FaceX.append(fx)
        self.FaceY.append(fy)

In [17]:
def extract_split(cases, accepted_o=[1, 2, 3, 4]):
    train, val, test = Dataset(), Dataset(), Dataset()
    
    i = 0
    for case in cases:
        dset = info_data(case)['Dataset']
        if not (case in invalid_cases):
            FRAME_N = frame_data(case)
            O = orientation_data(case)
            XCAM, YCAM = coordinate_data(case)
            FACE = get_face_info(case)
            
            if dset == 'train': dataset = train
            elif dset == 'val': dataset = val
            elif dset == 'test': dataset = test

            for frame_n, o, xcam, ycam, \
                fh, fw, fx, fy, valid in zip(FRAME_N, O, XCAM, YCAM, 
                                     FACE['H'], FACE['W'], FACE['X'], FACE['Y'], FACE['IsValid']):
                if o in accepted_o and valid:
                    dataset.append('{}/frames/{}'.format(case.name, frame_n), 
                                   xcam, 
                                   ycam,
                                   fh, fw, fx, fy)
        

    dset_df = []
    for dataset in [train, test, val]:
        # package to dataframe
        df = pd.DataFrame(data={'file_names': dataset.fnames, 
                                'XCam': dataset.XCam,
                                'YCam': dataset.YCam,
                                'FaceH': dataset.FaceH, 
                                'FaceW': dataset.FaceW,
                                'FaceX': dataset.FaceX,
                                'FaceY': dataset.FaceY})        
        dset_df.append(df)

    return dset_df[0], dset_df[1], dset_df[2]

## Portrait Split

In [18]:
traindf, valdf, testdf = extract_split(list(dset_path.iterdir()), accepted_o=[1])
traindf.to_csv('papersplit/portrait/traindf.csv')
valdf.to_csv('papersplit/portrait/valdf.csv')
testdf.to_csv('papersplit/portrait/testdf.csv')

In [23]:
traindf.sample(frac=1).head()

,file_names,XCam,YCam,FaceH,FaceW,FaceX,FaceY
254493,03495/frames/00614.jpg,0.699654,-7.244497,309.765119,310.069759,64.15312,273.82144
670077,02220/frames/00441.jpg,-0.822311,-2.067800,288.959999,288.959999,176.52000,290.52000
354688,02566/frames/00103.jpg,0.205354,-7.052441,228.085759,228.215679,107.25816,308.46912
37625,00114/frames/00016.jpg,-0.887413,-7.757504,210.560000,210.560000,150.72000,428.72000
471154,01930/frames/00216.jpg,3.327601,-7.571447,288.959999,289.470719,57.99664,339.60800


In [24]:
len(traindf), len(valdf), len(testdf)

(697315, 92575, 29538)

## Landscape Split

In [25]:
traindf, valdf, testdf = extract_split(list(dset_path.iterdir()), accepted_o=[3])
traindf.to_csv('papersplit/landscape-r/traindf.csv')
valdf.to_csv('papersplit/landscape-r/valdf.csv')
testdf.to_csv('papersplit/landscape-r/testdf.csv')

In [26]:
traindf, valdf, testdf = extract_split(list(dset_path.iterdir()), accepted_o=[4])
traindf.to_csv('papersplit/landscape-l/traindf.csv')
valdf.to_csv('papersplit/landscape-l/valdf.csv')
testdf.to_csv('papersplit/landscape-l/testdf.csv')